In [ ]:
# script to generate slightly perturbed models in RMG reaction order

In [1]:
import os
import sys
import subprocess


import numpy as np
import rmgpy.chemkin

import matplotlib.pyplot as plt
%matplotlib inline

In [49]:
# Load the base model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/rmg_model'
model_dir = '/scratch/harris.se/autoscience/local_sensitivity/models'  # where the output cti's go

base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)


In [4]:
def perturb_reaction(rxn, delta):
    # takes in an RMG reaction object
    # delta is the ln(k) amount to perturb the A factor
    # delta is a multiplicative factor- units don't matter, yay!
    # does not deepycopy because there's some issues with rmgpy.reactions copying

    rxn.kinetics.A.value *= np.exp(delta)

In [54]:
def perturb_species(species, delta):
    # takes in an RMG species object
    # change the enthalpy offset
    for poly in species.thermo.polynomials:
        new_coeffs = poly.coeffs
        new_coeffs[5] *= (1.0 + delta)
        poly.coeffs = new_coeffs

In [ ]:
def save_new_cti(param_index, reaction=True):
    # if false, we're perturbing a species
    
    # load a fresh model
    species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
    
    
    if reaction:
        model_index = param_index
        delta = 0.1
        try:
            perturb_reaction(reaction_list[param_index], delta)
        except AttributeError:
            print(f'skipping reaction {param_index} {type(reaction_list[param_index])}')
            return
    else:  # perturb the thermo
        model_index = param_index + len(reaction_list)
        delta = 0.1
        perturb_species(species_list[param_index], delta)
        
        
    # save the new mechanism
    chemkin_file = os.path.join(model_dir, f'chem_{model_index:04}.inp')
    rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
    subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={model_dir}/chem_{reaction_index:04}.cti'])
    os.remove(chemkin_file)

In [ ]:
for i in range(0, 5):
    save_new_cti(i)